<a href="https://colab.research.google.com/github/vijayshankarrealdeal/intro_to_pytorch-Gans/blob/main/First.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import  nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)


In [ ]:
loss_par = nn.BCEWithLogitsLoss()
n_epochs = 5000
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.00001
device = 'cuda'

In [ ]:
def show_tensor_image(image_tensor,num_images = 25,size = (1,28,28)):
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
def get_genrator_block(input_dim,output_dim):
  return nn.Sequential(
      nn.Linear(input_dim,output_dim),
      nn.BatchNorm1d(output_dim),
      nn.ReLU(inplace = True)
  )
  

In [ ]:
class Genrator(nn.Module):
  def __init__(self,z_dim = 10,img_dim = 28*28,hidden_units = 128):
    super(Genrator,self).__init__()
    self.gen = nn.Sequential(
            get_genrator_block(z_dim, hidden_units),
            get_genrator_block(hidden_units, hidden_units * 2),
            get_genrator_block(hidden_units * 2, hidden_units * 4),
            get_genrator_block(hidden_units * 4, hidden_units * 8),
            nn.Linear(hidden_units * 8,img_dim),
            nn.Sigmoid()
    )

  def forward(self,noise):
    return self.gen(noise)

  def get_gen(self):
    return self.gen


In [ ]:
def get_noise(n_samples,z_dim,device = device):
  noise = torch.randn(n_samples,z_dim,device = device)
  return noise

In [ ]:
def get_discriminator_block(input_dim,output_dim):
  return nn.Sequential(
      nn.Linear(input_dim,output_dim),
      nn.LeakyReLU(0.2)
  )

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,img_dim = 28*28,hidden_units = 128):
    super(Discriminator,self).__init__()
    self.dis = nn.Sequential(
         get_discriminator_block(img_dim, hidden_units * 4),
         get_discriminator_block(hidden_units * 4, hidden_units * 2),
         get_discriminator_block(hidden_units * 2, hidden_units),
         nn.Linear(hidden_units,1)
    )
    def forward(self,images):
      return self.dis(images)
    
    def get_disc(self):
      return self.dis
  


In [ ]:
gen = Genrator(z_dim).to(device)
gen_optimizer = torch.optim.Adam(gen.parameters(),lr = lr)
disc = Discriminator().to(device)
disc_optimizer = torch.optim.Adam(disc.parameters(),lr = lr)

In [ ]:
def discr_loss(gen, disc, criterion, real, num_samples, z_dim, device):
  noise = get_noise(num_samples,z_dim,device=device)
  fake = gen(noise).detach()
  fake_pred = disc(fake)
  real_pred = disc(real)
  print("I am okay")
  fake_loss = criterion(fake_pred,torch.zeros(num_images, 1).to(device))
  real_loss = criterion(real_pred,torch.ones(num_images, 1).to(device))
  loss = (fake_loss + real_loss)/2
 
  return loss

In [ ]:
def gen_loss(gen,dis,loss_par,num_samples,z_dim,device):
  noise = get_noise(num_samples,z_dim,device)
  gen_vec = gen(noise)
  dis_pred = dis(gen_vec)
  gen_loss = loss_par(dis_pred,torch.ones(dis_pred.shape,device=device))
  return gen_loss

In [ ]:
dataloader = DataLoader(
    MNIST('.', download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
cur_step = 0
mean_gen_loss = 0
mean_disc_loss = 0
test_generator = True
gen_loss = False
eroor = False
for i in range(n_epochs):
  for real,_ in tqdm(dataloader):
    cur_batch_size = len(real)
    real = real.view(cur_batch_size, -1).to(device)
    disc_optimizer.zero_grad()
    disc_loss = discr_loss(gen,disc,loss_par,real,cur_batch_size,z_dim,device)
    
    disc_loss.backward(retain_graph=True)
    disc_optimizer.step()
    if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

    gen_optimizer.zero_grad()
    gen_loss = gen_loss(gen,disc,loss_par,cur_batch_size,z_dim,device)
    gen_loss.backward(retain_graph = True)
    gen_optimizer.step()
    mean_disc_loss += disc_loss.item()/display_step
    mean_gen_loss += gen_loss.item()/display_step



    if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            show_tensor_images(fake)
            show_tensor_images(real)
            mean_generator_loss = 0
            mean_discriminator_loss = 0
    cur_step += 1


NotImplementedError: ignored